In [1]:
from wiggle_bin_simulation import load_and_process_emwcf_data, add_bin_models_air_soil, combine_series_by_label
from wiggle_bin_plot import plot_worm_temperature_zones, plot_hours_in_temperature_zones

In [2]:
base_models = {
    'bin_model_A': {
        'partial_shade': True,
        'color': 'black',
        'insulation': True,
        'auto_venting_lid': True,
        'shade_on_hot_days': True
    },
    'bin_model_B': {
        'partial_shade': True,
        'color': 'white',
        'insulation': True,
        'auto_venting_lid': True
    }
}

name_model_A = 'Black box'
name_model_B = 'White box'

plot_air_cols = {
    'temp': {'name': 'ERA5'},
    'bin_model_A_air': {'name': name_model_A},
    'bin_model_B_air': {'name': name_model_B}
}
plot_soil_cols = {
    'soil_0_7': {'name': 'ERA5'},
    'bin_model_A_soil': {'name': name_model_A},
    'bin_model_B_soil': {'name': name_model_B}
}

# 2019

In [3]:
year_2019 = load_and_process_emwcf_data(
    "emwcf/2019-2h-interval/instant.nc",
    "emwcf/2019-2h-interval/accum.nc"
)
year_2019 = add_bin_models_air_soil(year_2019, base_models)

In [4]:
fig = plot_worm_temperature_zones(year_2019, plot_air_cols, title="WiggleBin - 2019 - Air Temperature")
fig.show()

In [5]:
fig = plot_worm_temperature_zones(year_2019, plot_soil_cols, title="WiggleBin - 2019 - Soil Temperature")
fig.show()

In [6]:
series_to_compare = {
    'ERA5': year_2019['soil_0_7'],
    name_model_A: year_2019['bin_model_A_soil'],
    name_model_B: year_2019['bin_model_B_soil'],
}

fig = plot_hours_in_temperature_zones(series_to_compare, "WiggleBin - 2019 - Time in Worm Temperature Zones")
fig.show()

# 2021

In [7]:
year_2021 = load_and_process_emwcf_data(
    "emwcf/2021-2h-interval/instant.nc",
    "emwcf/2021-2h-interval/accum.nc"
)
year_2021 = add_bin_models_air_soil(year_2021, base_models)

In [8]:
fig = plot_worm_temperature_zones(year_2021, plot_air_cols, title="WiggleBin - 2021 - Air Temperature")
fig.show()

In [9]:
fig = plot_worm_temperature_zones(year_2021, plot_soil_cols, title="WiggleBin - 2021 - Soil Temperature")
fig.show()

In [10]:
series_to_compare = {
    name_model_A: year_2021['bin_model_A_soil'],
    name_model_B: year_2021['bin_model_B_soil'],
    'ERA5': year_2021['soil_0_7']
}

fig = plot_hours_in_temperature_zones(series_to_compare, "WiggleBin - 2021 - Time in Worm Temperature Zones")
fig.show()

# Combined years

In [11]:
dfs = [year_2019, year_2021]
labels = ['ERA5', name_model_A, name_model_B]
cols = ['soil_0_7', 'bin_model_A_soil', 'bin_model_B_soil']

series_to_compare_combined = combine_series_by_label(dfs, labels, cols)

fig = plot_hours_in_temperature_zones(series_to_compare_combined, "2019 + 2021 Time in Worm Temperature Zones")
fig.show()